In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/english-hindi-dataset/Sentence pairs in English-Hindi - 2025-02-11.tsv


In [6]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer
import torch
from indicnlp.tokenize import indic_tokenize

In [4]:
!pip3 install indic-nlp-library

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.4 MB/s eta 0:00:00


In [ ]:
data = pd.read_csv(
    "/kaggle/input/english-hindi-dataset/Sentence pairs in English-Hindi - 2025-02-11.tsv",
    sep="\t",
    header=None,
    names=["SrcSentID", "SrcSent", "DstSentID", "DstSent"]
)

In [ ]:
data.head()

In [ ]:
data.shape


In [ ]:

data.drop(labels=[data.columns[0],data.columns[2]],axis=1,inplace=True)


In [ ]:
data.head()

In [ ]:
src_sent_tokenizer = AutoTokenizer.from_pretrained("google-T5/T5-base")

In [ ]:
data["SrcSent"] = data["SrcSent"].apply(lambda x: src_sent_tokenizer.tokenize(x))


In [ ]:
data.head()

In [ ]:
data["DStSent"] = data["DstSent"].apply(lambda x: indic_tokenize.trivial_tokenize(x, lang="hi"))


In [ ]:
data.head()

In [ ]:
data["DstSent"] = data["DstSent"].apply(lambda x: indic_tokenize.trivial_tokenize(x,lang="hi"))

In [ ]:
data["SrcSent"] = data["SrcSent"].apply(src_sent_tokenizer.convert_tokens_to_ids)

In [ ]:
Vs = src_sent_tokenizer.get_vocab()

In [ ]:
len(Vs)

In [ ]:
data.head()

In [ ]:
hindi_vocab = set()

for tokenized_hindi_sent in data["DstSent"]:
    hindi_vocab.update(tokenized_hindi_sent)



In [ ]:
Vd=dict()

In [ ]:
Vd = dict()
for idx, token in enumerate(hindi_vocab):
    Vd[token] = idx + 3
Vd["<PAD>"] = 0
Vd["<SOS>"] = 1
Vd["<EOS>"] = 2


# Fir hindi_vocab ke tokens ko index dein



In [ ]:
data.head()

In [ ]:
def convert_hindi_tokens_to_ids(hindi_sent):
    return [Vd[token] for token in hindi_sent]

In [ ]:
data["DstSentLabel"] = data["DstSent"].apply(lambda x: insert_eos_token_id(x))

In [ ]:
data.head()

In [ ]:
def insert_sos_token_id(hindi_sent_token_ids):
    return [1] + hindi_sent_token_ids

In [ ]:
data["DstSentInput"] = data["DstSent"].apply(lambda x: insert_sos_token_id(x))

In [ ]:
def insert_eos_token_id(hindi_sent_token_ids):
    return hindi_sent_token_ids + [2]

In [ ]:
data["DstSentLabel"] = data["DstSent"].apply(lambda x: insert_eos_token_id(x))

In [ ]:
data.drop(labels=[data.columns[1]],axis=1,inplace=True)

In [ ]:
data.head()

In [ ]:
X = list(data["SrcSent"])
Y_input = list(data["DstSentInput"])
Y_label = list(data["DstSentLabel"])

In [ ]:
X_tensor = [torch.tensor(tokenized_eng_sent_ids) for tokenized_eng_sent_ids in X]
Y_input_tensor = [torch.tensor(tokenized_hin_sent_ids) for tokenized_hin_sent_ids in Y_input]
Y_label_tensor = [torch.tensor(tokenized_hin_sent_ids) for tokenized_hin_sent_ids in Y_label]

In [ ]:
X_padded = torch.nn.utils.rnn.pad_sequence(X_tensor,batch_first=True)
Y_padded_input = torch.nn.utils.rnn.pad_sequence(Y_input_tensor,batch_first=True)
Y_padded_label = torch.nn.utils.rnn.pad_sequence(Y_label_tensor,batch_first=True)

In [ ]:
 Ns = X_padded.shape[1]
Nd = Y_padded_label.shape[1]

In [11]:
class Encoder(torch.nn.Module):
    def __init__(self,src_lang_vocab_size,word_embedding_dim):
        super(Encoder,self).__init__()
        self.first_embedding_layer = torch.nn.Embedding(num_embeddings=src_lang_vocab_size,embedding_dim=word_embedding_dim)
        self.second_lstm_layer = torch.nnLSTM(input_size==word_embedding_dim,hidden_size=word_embedding_dim,
                                             batch_first=True)
        def forward(self,x_padded_mini_batch):
            first_embedding_layer_out=self.first_embedding_layer(x_padded_mini_batch)
            encoder_output,(final_encoder_output,final_cell_state)=self.second_1stm_layer(first_embedding_layer_out)

            return encoder_output,(final_encoder_output,final_cell_state)

In [ ]:
class Decoder(torch.nn.Module):

    def __init__(self,dst_lang_vocab_size,word_embedding_dim):
        super(Decoder,self).__init__()

        self.first_embedding_layer = torch.nn.Embedding(num_embeddings=dst_lang_vocab_size,
                                                       embedding_dim=word_embedding_dim)
        self.second_lstm_layer = torch.nn.LSTM(input_size=word_embedding_dim,
                                               hidden_size=word_embedding_dim,
                                              batch_first=True)
        self.prediction_layer = torch.nn.Linear(in_feature=word_embedding_dim,out_features=dst_lang_vocab_size)
        self.prediction_layer_activation=torch.nn.Softmax(dim=2)

    def forward(self,Y_padded_input_mini_batch,final_encoder_output,final_cell_state):

                          final_encoder_output,
        